In [1]:
import json

with open('config.json', 'r') as f:
    config = json.load(f)

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA

In [3]:
df_news_clusterizada = pd.read_csv(config["DF_ITEMS_CLUSTERED_FEATURES"])

In [4]:
df_news_clusterizada.head()

,Unnamed: 0.1,Unnamed: 0,page,url,issued,modified,title,body,caption,cleaned_body,cleaned_title,cleaned_caption,combined_text,cluster,word_count
0,0,0,7371a9b5-5824-4c57-8704-00a74feebe79,http://g1.globo.com/al/alagoas/noticia/2018/09...,2018-09-13 14:52:55+00:00,2018-09-14 16:14:49+00:00,Corpo de motorista da Uber é encontrado em can...,Corpo de motorista de aplicativo desaparecido ...,"Segundo a polícia, Antônio Vitor foi solicitad...",corpo motorista aplicativo desaparecido encont...,corpo motorista uber encontrado canavial rotei...,polícia antônio vitor solicitado corridas desa...,corpo motorista uber encontrado canavial rotei...,0,234
1,1,1,7a5ea08f-4583-49e2-ba52-a71999443f7b,http://g1.globo.com/am/amazonas/noticia/detent...,2018-05-20 20:42:40+00:00,2018-05-20 20:42:40+00:00,Detento recapturado após fuga por túnel volta ...,Detento disse que passou nome falso ao dar ent...,Ele tinha registro em presídio com nome falso....,detento disse passou nome falso entrada presíd...,detento recapturado fuga túnel volta cdpm manaus,tinha registro presídio nome falso presos esca...,detento recapturado fuga túnel volta cdpm mana...,0,345
2,2,2,6afc8bbb-4f36-43d5-8a44-a2917df5621a,http://g1.globo.com/ap/amapa/noticia/audios-mo...,2017-07-30 00:37:17+00:00,2017-07-30 00:48:42+00:00,Áudios mostram conversa entre bandidos durante...,Áudios mostram possível conversa entre bandido...,Revista realizada na sexta-feira (28) no Iapen...,áudios mostram conversa bandidos tentativa fug...,áudios mostram conversa bandidos durante tenta...,revista realizada iapen apreendeu celulares dr...,áudios mostram conversa bandidos durante tenta...,0,383
3,3,3,5cc3bd27-80c7-457d-a807-2e8e7fddf031,http://g1.globo.com/ap/amapa/noticia/2020/11/0...,2020-11-06 12:54:00+00:00,2020-11-12 21:22:52+00:00,FOTOS: Apagão no Amapá,"Moradores da capital do Amapá, em Macapá, faze...",Incêndio em subestação de energia deixa 13 dos...,moradores capital amapá macapá protestos maksu...,fotos apagão amapá,incêndio subestação energia deixa municípios e...,fotos apagão amapá incêndio subestação energia...,0,470
4,4,4,d6956177-db96-42f5-9f68-dd0d6e930661,http://g1.globo.com/ap/amapa/noticia/2019/05/2...,2019-05-27 13:43:03+00:00,2019-05-27 18:19:06+00:00,Profissionais da educação no AP paralisam ativ...,Profissionais da educação paralisam atividades...,Ato comprometeu aulas em escolas nesta segunda...,profissionais educação paralisam atividades re...,profissionais educação ap paralisam atividades...,ato comprometeu aulas escolas sinsepeap seed n...,profissionais educação ap paralisam atividades...,0,609


In [5]:
import numpy as np

# Convert the string column to datetime
df_news_clusterizada["data"] = pd.to_datetime(df_news_clusterizada["issued"])

In [6]:
df_news_clusterizada.head()

,Unnamed: 0.1,Unnamed: 0,page,url,issued,modified,title,body,caption,cleaned_body,cleaned_title,cleaned_caption,combined_text,cluster,word_count,data
0,0,0,7371a9b5-5824-4c57-8704-00a74feebe79,http://g1.globo.com/al/alagoas/noticia/2018/09...,2018-09-13 14:52:55+00:00,2018-09-14 16:14:49+00:00,Corpo de motorista da Uber é encontrado em can...,Corpo de motorista de aplicativo desaparecido ...,"Segundo a polícia, Antônio Vitor foi solicitad...",corpo motorista aplicativo desaparecido encont...,corpo motorista uber encontrado canavial rotei...,polícia antônio vitor solicitado corridas desa...,corpo motorista uber encontrado canavial rotei...,0,234,2018-09-13 14:52:55+00:00
1,1,1,7a5ea08f-4583-49e2-ba52-a71999443f7b,http://g1.globo.com/am/amazonas/noticia/detent...,2018-05-20 20:42:40+00:00,2018-05-20 20:42:40+00:00,Detento recapturado após fuga por túnel volta ...,Detento disse que passou nome falso ao dar ent...,Ele tinha registro em presídio com nome falso....,detento disse passou nome falso entrada presíd...,detento recapturado fuga túnel volta cdpm manaus,tinha registro presídio nome falso presos esca...,detento recapturado fuga túnel volta cdpm mana...,0,345,2018-05-20 20:42:40+00:00
2,2,2,6afc8bbb-4f36-43d5-8a44-a2917df5621a,http://g1.globo.com/ap/amapa/noticia/audios-mo...,2017-07-30 00:37:17+00:00,2017-07-30 00:48:42+00:00,Áudios mostram conversa entre bandidos durante...,Áudios mostram possível conversa entre bandido...,Revista realizada na sexta-feira (28) no Iapen...,áudios mostram conversa bandidos tentativa fug...,áudios mostram conversa bandidos durante tenta...,revista realizada iapen apreendeu celulares dr...,áudios mostram conversa bandidos durante tenta...,0,383,2017-07-30 00:37:17+00:00
3,3,3,5cc3bd27-80c7-457d-a807-2e8e7fddf031,http://g1.globo.com/ap/amapa/noticia/2020/11/0...,2020-11-06 12:54:00+00:00,2020-11-12 21:22:52+00:00,FOTOS: Apagão no Amapá,"Moradores da capital do Amapá, em Macapá, faze...",Incêndio em subestação de energia deixa 13 dos...,moradores capital amapá macapá protestos maksu...,fotos apagão amapá,incêndio subestação energia deixa municípios e...,fotos apagão amapá incêndio subestação energia...,0,470,2020-11-06 12:54:00+00:00
4,4,4,d6956177-db96-42f5-9f68-dd0d6e930661,http://g1.globo.com/ap/amapa/noticia/2019/05/2...,2019-05-27 13:43:03+00:00,2019-05-27 18:19:06+00:00,Profissionais da educação no AP paralisam ativ...,Profissionais da educação paralisam atividades...,Ato comprometeu aulas em escolas nesta segunda...,profissionais educação paralisam atividades re...,profissionais educação ap paralisam atividades...,ato comprometeu aulas escolas sinsepeap seed n...,profissionais educação ap paralisam atividades...,0,609,2019-05-27 13:43:03+00:00


In [7]:


# Find the most recent date
most_recent_date = df_news_clusterizada["data"].max()

# Calculate the difference in days
df_news_clusterizada["idade_em_dias"] = (most_recent_date - df_news_clusterizada["data"]).dt.days

# Assign weights using exponential decay (you can adjust the base for scaling)
df_news_clusterizada["peso"] = np.exp(-df_news_clusterizada["idade_em_dias"] / 30)  # 30 days as the decay factor

In [8]:
most_recent_date

Timestamp('2022-08-15 02:31:41+0000', tz='UTC')

In [9]:
# Adjust range to [0.5, 1]
new_min = 0.1
new_max = 1
df_news_clusterizada["peso_ajustado"] = df_news_clusterizada["peso"] * (new_max - new_min) + new_min


In [10]:
df_news_clusterizada.head(50)

,Unnamed: 0.1,Unnamed: 0,page,url,issued,modified,title,body,caption,cleaned_body,cleaned_title,cleaned_caption,combined_text,cluster,word_count,data,idade_em_dias,peso,peso_ajustado
0,0,0,7371a9b5-5824-4c57-8704-00a74feebe79,http://g1.globo.com/al/alagoas/noticia/2018/09...,2018-09-13 14:52:55+00:00,2018-09-14 16:14:49+00:00,Corpo de motorista da Uber é encontrado em can...,Corpo de motorista de aplicativo desaparecido ...,"Segundo a polícia, Antônio Vitor foi solicitad...",corpo motorista aplicativo desaparecido encont...,corpo motorista uber encontrado canavial rotei...,polícia antônio vitor solicitado corridas desa...,corpo motorista uber encontrado canavial rotei...,0,234,2018-09-13 14:52:55+00:00,1431,1.923770e-21,0.100000
1,1,1,7a5ea08f-4583-49e2-ba52-a71999443f7b,http://g1.globo.com/am/amazonas/noticia/detent...,2018-05-20 20:42:40+00:00,2018-05-20 20:42:40+00:00,Detento recapturado após fuga por túnel volta ...,Detento disse que passou nome falso ao dar ent...,Ele tinha registro em presídio com nome falso....,detento disse passou nome falso entrada presíd...,detento recapturado fuga túnel volta cdpm manaus,tinha registro presídio nome falso presos esca...,detento recapturado fuga túnel volta cdpm mana...,0,345,2018-05-20 20:42:40+00:00,1547,4.026069e-23,0.100000
2,2,2,6afc8bbb-4f36-43d5-8a44-a2917df5621a,http://g1.globo.com/ap/amapa/noticia/audios-mo...,2017-07-30 00:37:17+00:00,2017-07-30 00:48:42+00:00,Áudios mostram conversa entre bandidos durante...,Áudios mostram possível conversa entre bandido...,Revista realizada na sexta-feira (28) no Iapen...,áudios mostram conversa bandidos tentativa fug...,áudios mostram conversa bandidos durante tenta...,revista realizada iapen apreendeu celulares dr...,áudios mostram conversa bandidos durante tenta...,0,383,2017-07-30 00:37:17+00:00,1842,2.159329e-27,0.100000
3,3,3,5cc3bd27-80c7-457d-a807-2e8e7fddf031,http://g1.globo.com/ap/amapa/noticia/2020/11/0...,2020-11-06 12:54:00+00:00,2020-11-12 21:22:52+00:00,FOTOS: Apagão no Amapá,"Moradores da capital do Amapá, em Macapá, faze...",Incêndio em subestação de energia deixa 13 dos...,moradores capital amapá macapá protestos maksu...,fotos apagão amapá,incêndio subestação energia deixa municípios e...,fotos apagão amapá incêndio subestação energia...,0,470,2020-11-06 12:54:00+00:00,646,4.448280e-10,0.100000
4,4,4,d6956177-db96-42f5-9f68-dd0d6e930661,http://g1.globo.com/ap/amapa/noticia/2019/05/2...,2019-05-27 13:43:03+00:00,2019-05-27 18:19:06+00:00,Profissionais da educação no AP paralisam ativ...,Profissionais da educação paralisam atividades...,Ato comprometeu aulas em escolas nesta segunda...,profissionais educação paralisam atividades re...,profissionais educação ap paralisam atividades...,ato comprometeu aulas escolas sinsepeap seed n...,profissionais educação ap paralisam atividades...,0,609,2019-05-27 13:43:03+00:00,1175,9.775361e-18,0.100000
5,5,5,a3254b68-978d-4a28-b650-f59e10837def,http://g1.globo.com/ap/amapa/noticia/2019/04/1...,2019-04-12 10:19:31+00:00,2019-04-12 13:14:16+00:00,"Após 15 anos de obras e paralisações, novo aer...",Novo terminal de passageiros vai atender até 5...,Nova estrutura deve dobrar capacidade do atual...,terminal passageiros atender milhões viajantes...,anos obras paralisações aeroporto macapá será ...,estrutura dobrar capacidade atual podendo rece...,anos obras paralisações aeroporto macapá será ...,0,766,2019-04-12 10:19:31+00:00,1220,2.181178e-18,0.100000
6,6,6,fc7a68a2-2ab5-4c8b-898b-e9c94050c477,http://g1.globo.com/ap/amapa/noticia/2021/10/2...,2021-10-27 14:19:29+00:00,2021-10-27 14:19:30+00:00,Prova do concurso dos Bombeiros do Amapá é anu...,Candidatos de concurso público do Corpo de Bom...,Seleção para sargentos da corporação foi alvo ...,candidatos concurso público corpo bombeiros am...,prova concurso bombeiros amapá anulada será re...,seleção sargentos corporação alvo fraude inves...,prova concurso bombeiros amapá anulada será re...,0,372,2021-10-27 14:19:29+00:00,291,6.128350e-05,0.1

In [11]:
df_news_clusterizada.to_csv(config["DF_ITEMS_CLUSTERED_FEATURES_ADJ"])